# Data science tips
## 케라스 API를 사용한 사용자 정의 모델 만들기

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div> 

<div style="text-align: right"> Initial upload: 2020.11.29</div> 
<div style="text-align: right"> Last update: 2020.11.29</div> 

- 참고자료  
    - https://www.youtube.com/watch?v=wzxh3wg5Ysw&feature=youtu.be
    - https://github.com/rickiepark/handson-ml2/blob/master/custom_model_in_keras.ipynb

2020 대전 러닝 데이(DLD)에 소개된 내용을 정라한 내용이다.

### 1. 라이브러리 import, 데이터 읽기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import datetime
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.4.0-dev20200730'

- tf 2.3 버전 이상에서 사용가능

MNIST 데이터셋 사용

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
X_train = X_train.reshape(-1, 784)/255.

In [6]:
X_train.shape

(60000, 784)

### 2. Sequential 모델 만들기

In [8]:
seq_model = tf.keras.Sequential()
seq_model.add(tf.keras.layers.Dense(units = 10, activation='softmax', input_shape= (784,)))

In [9]:
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [10]:
seq_model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# 레이블을 0 ~ 9 사이의 값으로 해줬기 때문에 loss를 'sparse_categorical_crossentropy'로 지정

In [11]:
seq_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6792 - accuracy: 0.8203
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3165 - accuracy: 0.9119


### 3. 함수형 API 만들기

In [15]:
inputs = tf.keras.layers.Input(784)
outputs = tf.keras.layers.Dense(units=10, activation='softmax')(inputs) # __call()__ 메서드 호출

func_model = tf.keras.Model(inputs, outputs)

In [16]:
func_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [17]:
func_model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
func_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6618 - accuracy: 0.8295
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3062 - accuracy: 0.9141


Input 살펴보기

In [20]:
type(tf.keras.layers.Input)

function

- Dense층에 input 정보를 넣어주는 역할... 입력층이라기 보다는 입력 그 자체임... 성능에 영향 없음

따라서 다음 두 명령은 같다.  

In [21]:
# inputs = tf.keras.layers.Input(784)

In [23]:
input_layer = tf.keras.layers.InputLayer(784)
inputs = input_layer._inbound_nodes[0].outputs

In [24]:
outputs = tf.keras.layers.Dense(units=10, activation='softmax')(inputs)
input_layer_model = tf.keras.Model(inputs, outputs)

In [25]:
input_layer_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [26]:
input_layer_model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
input_layer_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6557 - accuracy: 0.8354
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3033 - accuracy: 0.9184


함수형 API를 사용한 모델은 layers 속성에 InputLayer클래스를 포함한다.(Sequential 모델과 차이점)

In [28]:
seq_model.layers

- 확인안됨. 
- 하지만 seq_model도 InputLayer 속성을 사용한다.(숨겨져 있다.) _layers 속성으로 확인 가능하다. 

In [31]:
seq_model._layers

_input_layers 속성에서도 확인 가능

In [32]:
seq_model._input_layers, func_model._input_layers

([<tensorflow.python.keras.engine.input_layer.InputLayer at 0x23600968508>],
 [<tensorflow.python.keras.engine.input_layer.InputLayer at 0x236011b4248>])

In [33]:
seq_model._output_layers, func_model._output_layers

([<tensorflow.python.keras.layers.core.Dense at 0x2360087bf48>],
 [<tensorflow.python.keras.layers.core.Dense at 0x23600dac7c8>])

Model 클래스로 만든 func_model은 Functional 클래스의 객체이다.  
Model 클래스는 만들어진 클래를 Functional 클래스로 반환하는 역할을 한다.

In [34]:
func_model.__class__

tensorflow.python.keras.engine.functional.Functional

- 상속관계를 보면 Model =>(상속) Functional =>(상속) Sequential 형태이다.

### 4. 사용자 정의층 만들기

tf.layers.Layer 클래스를 상속하고 build() 메서드에 가중치를 만든다음 call() 메서드에서 연산을 구현한다.

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation = None, **kwargs)
        # units와 activation 매개변수 외에 나머지 변수를 부모 클래스의 생성자로 전달
        